<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Mohammadreza Ahmadi Teshnizi
### Student Number : 98170646
### Colab Link: https://colab.research.google.com/drive/177FbWUiuwHFiYOJ3W4PYjt6Hr135MmAg
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [2]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-08 16:51:35--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-08 16:51:35--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf3af3dcb4143bd36e17983d152.dl.dropboxusercontent.com/cd/0/get/B7pb9fgXEy7MQy57fe3HKkbSqNlR90naSgA_SYJh7OihnaLN1gEOHg1Y0W5lI4MX5XQ8PCa46zw-b8eF3psHUE9gyrWemk9bqR-lou25DLrwJqaCPw72FApKbRcDjyTMsr--6XYd6kv8NbceeQJsKHuEIPbf_OiZkbPk0fNFctNRLTEUloI5y4ZHzTd5ZWbNlJw/file?dl=1# [following]
--2023-05-08 16:51:36--  https://ucf3af3dcb4143bd36e17983d152.dl.dropboxusercontent.com/cd/0/get/B7pb9fgXEy7MQy57fe3HKkbSqNlR90naSgA_SYJ

In [3]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

Archive:  /content/dataset.zip
  inflating: /content/dataset/content/news-train.csv  


# Preprocessing

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

In [5]:
df = pd.read_csv('/content/dataset/content/news-train.csv')

# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [6]:
grouped = df.groupby('topic').count()
print(grouped)

             text
topic            
cultural    13359
economics   49600
politics    11297
sports     123105


Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [45]:
min_topic = df['topic'].value_counts().idxmin()
min_count = df['topic'].value_counts().min()

new_df = pd.DataFrame(columns=df.columns)
for topic in df['topic'].unique():
    if topic == min_topic:
        new_df = pd.concat([new_df, df[df['topic'] == topic]])
    else:
        new_df = pd.concat([new_df, df[df['topic'] == topic].sample(min_count, random_state=42)])

new_df = new_df.sample(frac=1, random_state=42)
print(new_df)


            topic                                               text
4615     politics  اگر ما با مردم آنچه هست بگوییم آنچه نیست # و ن...
158663  economics  رئیس سازمان چای کشور گفت صندوق حمایت از توسعه ...
99757    cultural  یک حرکت # برای حمایت از تولیدات ادبی و هنری نو...
151431   politics  عصر روز گذشته # ایران و روسیه در گفت‌وگویی تلف...
46625    politics  محمدرضا عارف روز گذشته در دیدار جمعی از # شورا...
...           ...                                                ...
92968      sports  به گزارش ایسنا جام‌جهانی # روسیه # روز دیگر آغ...
169808   cultural  نمایش کمدی # به کارگردانی میثم عبدی از # دی در...
157638   cultural  معاون پژوهشی جهاد دانشگاهی واحد استان البرز از...
118852     sports  به گزارش ورزش سه آبی پوشان پایتخت که در نیم فص...
79965    politics  عبدالله # فعال سیاسی اصلاح‌طلب در پاسخ به این ...

[45188 rows x 2 columns]


# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [8]:
train_df, test_df = train_test_split(new_df, test_size=0.2, random_state=42)

In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [12]:
def tokenize(text):
    return text.split(' ')

tokenized_train_df = train_df.copy()
tokenized_train_df['tokens'] = train_df['text'].apply(tokenize)

Count the tokens and list the top 5 frequent ones (5 points)

In [13]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df['text'])

tokens = vectorizer.get_feature_names_out()

token_counts = X_train.sum(axis=0)

freq_tokens = [(token, count) for token, count in zip(tokens, token_counts.tolist()[0])]
freq_tokens = sorted(freq_tokens, key=lambda x: x[1], reverse=True)[:5]

print("Top 5 frequent tokens:")
for token, count in freq_tokens:
    print(f"{token}: {count}")



Top 5 frequent tokens:
در: 744170
به: 609625
از: 435555
که: 425924
این: 423838


Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [14]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-08 17:13:55--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-08 17:13:55--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [text/plain]
Saving to: ‘/content/stopwords.txt’

/content/stopwords. 100%[===================>]   2.87K  --.-KB/s    in 0s      

2023-05-08 17:13:56 (29.1 MB/

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

with open("/content/stopwords.txt", encoding="utf8") as f:
    stopwords = f.read().splitlines()

stopwords.append("#")


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['اثر', 'البت', 'بالای', 'برابر', 'برای', 'بیرون', 'تول', 'توی', 'تی', 'جلوی', 'حدود', 'خارج', 'دنبال', 'روی', 'زیر', 'سری', 'سمت', 'سوی', 'ضد', 'طبق', 'عقب', 'عل', 'عنوان', 'قصد', 'لطفا', 'مد', 'نزد', 'نزدیک', 'وسط', 'پاعین', 'کنار'] not in stop_words.
  warnings.warn(


Make a list of the remaining unique tokens (5 points)

In [46]:
tokens = vectorizer.vocabulary_.keys()
tokens_without_stopwords = [token for token in tokens if token not in stopwords and token != '#']
unique_tokens = list(set(tokens_without_stopwords))
print(unique_tokens)

['رسانی', 'غلامی', 'پایگاه', 'ایام', 'برسیم', 'مادرید', 'کنندگان', 'کارگردان', 'تاسیسات', 'صاحبان', 'معاملات', 'میانگین', 'کارگروه', 'ماده', 'سایه', 'باقری', 'دنیای', 'عملکردی', 'کشت', 'حاصل', 'درخت', 'اکنون', 'نشود', 'عملکرد', 'نباشد', 'محبوب', 'بگوید', 'تایلند', 'صربستان', 'جبران', 'سویا', 'ایستادگی', 'جلالی', 'پله', 'مشکلاتی', 'روشنی', 'لندن', 'بشر', 'خودداری', 'استانبول', 'پرونده', 'موانع', 'همیشه', 'توجه', 'نامزدها', 'دانشکده', 'متصل', 'پاریس', 'تعطیل', 'خواند', 'ساماندهی', 'حاضر', 'بیانیه', 'چاره', 'دروغ', 'جعفری', 'موضع', 'تبریزی', 'مس', 'سازوکار', 'کردی', 'سقف', 'گرت', 'چرخه', 'فراوانی', 'بازرگان', 'کرج', 'مایل', 'پهلوانی', 'نهادهای', 'گذر', 'اتحاد', 'شریف', 'مشتری', 'شیاطین', 'فردی', 'بالاتر', 'کمیسیون', 'طلبان', 'یکپارچه', 'بابک', 'خروج', 'وعده', 'غیاب', 'اساسی', 'پانزدهم', 'مند', 'شادی', 'دیده', 'گام', 'شیمیایی', 'بندرعباس', 'ذیل', 'توییتر', 'مصدق', 'پاس', 'معیشتی', 'ورود', 'کنی', 'الاسلام', 'قاسم', 'درآمد', 'رختکن', 'دستگیر', 'دلایلی', 'تومان', 'برون', 'قادر', 'بازار', 'سمت

Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [49]:
vectorizer = CountVectorizer(stop_words=stopwords)
X_train = vectorizer.fit_transform(train_df['text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['اثر', 'البت', 'بالای', 'برابر', 'برای', 'بیرون', 'تول', 'توی', 'تی', 'جلوی', 'حدود', 'خارج', 'دنبال', 'روی', 'زیر', 'سری', 'سمت', 'سوی', 'ضد', 'طبق', 'عقب', 'عل', 'عنوان', 'قصد', 'لطفا', 'مد', 'نزد', 'نزدیک', 'وسط', 'پاعین', 'کنار'] not in stop_words.
  warnings.warn(


In [53]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0]])

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [26]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['topic'])
print(y_train)

[3 3 1 ... 1 0 3]


Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [29]:
vectorizer_test = CountVectorizer(vocabulary=unique_tokens, stop_words=stopwords)

x_test = vectorizer_test.fit_transform(test_df['text']).toarray()

print(x_test)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['اثر', 'البت', 'بالای', 'برابر', 'برای', 'بیرون', 'تول', 'توی', 'تی', 'جلوی', 'حدود', 'خارج', 'دنبال', 'روی', 'زیر', 'سری', 'سمت', 'سوی', 'ضد', 'طبق', 'عقب', 'عل', 'عنوان', 'قصد', 'لطفا', 'مد', 'نزد', 'نزدیک', 'وسط', 'پاعین', 'کنار'] not in stop_words.
  warnings.warn(


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 7]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [55]:
y_test = label_encoder.transform(test_df['topic'])
print(y_test)

[2 3 0 ... 3 0 3]


# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [56]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(x_train, y_train)


MultinomialNB()

Get the predictions of the model for the testing news (`x_test`).

In [57]:
y_pred = nb.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [58]:
from sklearn.metrics import classification_report
number_to_topic = {v: k for k, v in topic_to_number.items()}
print(classification_report(y_test, y_pred, target_names=number_to_topic.values()))


              precision    recall  f1-score   support

      sports       0.19      0.07      0.10      2313
   economics       0.27      0.29      0.28      2253
    cultural       0.24      0.04      0.06      2262
    politics       0.24      0.60      0.34      2210

    accuracy                           0.24      9038
   macro avg       0.24      0.25      0.20      9038
weighted avg       0.24      0.24      0.20      9038



## Using your own code (15 points)

In [59]:
class Naive_Bayes:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        """
        Initializes the class.
        """
        self.classes_ = None
        self.class_prior_ = None
        self.feature_count_ = None
        self.feature_log_prob_ = None
        self.eps = 1e-9
    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        # Count the number of occurrences of each feature for each class
        self.classes_, y_indices = np.unique(y, return_inverse=True)
        self.class_count_ = np.bincount(y_indices)
        self.feature_count_ = np.zeros((len(self.classes_), X.shape[1]), dtype=np.float64)
        for i, j in enumerate(y_indices):
            self.feature_count_[j, :] += X[i, :]

        # Calculate the class prior probabilities
        self.class_prior_ = self.class_count_ / self.class_count_.sum()

        # Calculate the feature log probabilities
        smoothed_fc = self.feature_count_ + self.eps
        smoothed_cc = smoothed_fc.sum(axis=1)
        self.feature_log_prob_ = (np.log(smoothed_fc) -
                                  np.log(smoothed_cc.reshape(-1, 1)))

        return self
    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted value
        """
        
        n_samples, n_features = X.shape
        y_pred = np.zeros(n_samples, dtype=np.int64)
        
        log_prob = (np.log(self.class_prior_) +
                    X @ self.feature_log_prob_.T)
        y_pred = self.classes_[np.argmax(log_prob, axis=1)]

        return y_pred


Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [63]:
my_model = Naive_Bayes()
my_model.fit(x_train, y_train)


Get the predictions of your model for the testing news (`x_test`).

In [66]:
y_pred = my_model.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [67]:
from sklearn.metrics import classification_report

target_names = list(topic_to_number.keys())
print(classification_report(y_test, y_pred, target_names=target_names))


              precision    recall  f1-score   support

      sports       0.27      0.10      0.15      2313
   economics       0.11      0.00      0.00      2253
    cultural       0.26      0.87      0.40      2262
    politics       0.18      0.05      0.08      2210

    accuracy                           0.26      9038
   macro avg       0.20      0.26      0.16      9038
weighted avg       0.20      0.26      0.16      9038



# Kaggle

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef